# Operacions d'anàlisi amb pandas

## Operacions de lectura de dades

pandas proporciona diverses funcions per generar un dataframe a partir d'un arxiu de dades, amb diferents tipus de format:
`read_csv` per a fitxers CSV, `read_excel` per a fitxers Excel,
`read_json` per a fitxers JSON o `read_xml` per a fitxers XML són els més habituals.
Per referenciar el fitxer pot emprar-se un path relatiu o bé una URL.

Anam a treballar amb el conjunt de dades dels municipis de les Illes Balears, que podem trobar al Catàleg de Dades Obertes del Govern de les Illes Balears. L'enllaç per descarregar-lo en CSV és https://catalegdades.caib.cat/api/views/wwhu-ywi8/rows.csv

In [ ]:
import pandas as pd

# municipis = pd.read_csv('https://catalegdades.caib.cat/api/views/wwhu-ywi8/rows.csv') # ? Old, not working
municipis = pd.read_csv('https://opendata-ajuntament.barcelona.cat/data/cataleg.csv?public=true')
municipis.head()

## Operacions amb columnes

Entre d'altres, podem eliminar les columnes que no ens interessen amb el mètode DataFrame.drop. Ho farem amb beginLifespanVersion, endLifespanVersion, CODNUT3 i the_geom.
Una manera és especificant amb axis=1 que ens referim a columnes.
L'altra és especificant l'argument columnes.

En qualsevol dels dos casos, com és habitual en pandas, l'operació no té efecte sobre el dataframe original.
És per això que, si volem modificar el dataframe, li hem d'assignar el resultat del mètode, o bé hem d'utilitzar l'argument inplace=True.

In [ ]:

# ? Replace beginLifespanVersion & endLifespanVersion with title_es & title_en for the new csv
# Una manera d'esborrar columnes (amb axis=1 i assignació):
# municipis = municipis.drop(['beginLifespanVersion', 'endLifespanVersion'], axis=1)
municipis = municipis.drop(['title_es', 'title_en'], axis=1)

# ? Replace CODNUT3 & the_geom with 2022 & 2023 for the new csv
# I una altra (amb els arguments columns i inplace):
# municipis.drop(columns=['CODNUT3', 'the_geom'], inplace=True)
municipis.drop(columns=['2022', '2023'], inplace=True)

""" 
	If the key does not exists, KeyError will be thrown so may be convenient to wrap it with try-except
"""

municipis.head()

Una altra manera és seleccionar només les columnes que ens interessen i en quin ordre. Suposem que només ens interessen el nom, l'illa i l'àrea en hectàrees.

In [ ]:
# municipis = municipis[['NOM', 'NOM_CONSELL_ILLA', 'AREA_HA']] # ? Old, replace with name, date_published and package_count columns
municipis = municipis[['name', 'date_published', 'package_count']]
municipis.head()

I també podem afegir noves columnes, entre d'altres maneres, mitjançant el mètode DataFrame.insert. Hem d'especificar la posició de la nova columna, el seu nom, la llista amb els seus valors (o un valor per defecte per a tots) i es permeten valors repetits (per defecte, sí).
Afegirem una columna CAPITAL, després del nom de l'illa i li donarem el valor escalar False a totes les cel·les d'aquesta columna.

In [ ]:
municipis.insert(3, 'on_going', False, True)
municipis.head()


I ara li posarem True a la cel·la corresponent a Palma. És la fila que té índex 47, ja veurem més tard com fer seleccions.

In [ ]:
municipis.loc[1,'on_going'] = True
municipis.loc[1]

Per últim, podem canviar el nom d'una columna mitjançant DataFrame.rename. Aquí també hem d'assignar el resultat al dataframe o bé utilitzar inplace=True

In [ ]:
# municipis = municipis.rename(columns={'NOM_CONSELL_ILLA': 'ILLA', 'AREA_HA': 'HECTAREES'})
municipis = municipis.rename(columns={'date_published': 'date'})
municipis.head()

## Operacions estadístiques
En el lliurament 2 de Sistemes de Big Data es veuen diversos indicadors estadístics. Allà hem emprat fonamentalment NumPy, perquè hem fet feina amb dades unidimensionals. Però també podem emprar pandas: tant Series com DataFrame proporcionen un conjunt de mètodes que fan càlculs estadístics.
Vegem, per exemple, quina és la superfície mitjana dels municipis, mitjançant el mètode mean.

In [ ]:
# municipis['HECTAREES'].mean()
municipis['package_count'].mean()

Vegem la desviació estàndard (mètode std) i la variància (var) d'aquesta columna HECTAREES.

In [ ]:
# print('Desviació estàndard: {0}, variància: {1}'.format(municipis['HECTAREES'].std(), municipis['HECTAREES'].var()))
print('Desviació estàndard: {0}, variància: {1}'.format(municipis['package_count'].std(), municipis['package_count'].var()))

Els valors mínim i màxim:

In [ ]:
# print('Mínim: {0}, màxim: {1}'.format(municipis['HECTAREES'].min(), municipis['HECTAREES'].max()))
print('Mínim: {0}, màxim: {1}'.format(municipis['package_count'].min(), municipis['package_count'].max()))

I la superfície total de tots els municipis

In [ ]:
# municipis['HECTAREES'].sum()
municipis['package_count'].sum()

## Operacions de selecció
Podem seleccionar un conjunt de files que satisfacin una condició.
Ho feim de la forma:

`dataframe[condició]`

Per exemple, si volem seleccionar el municipi de Palma, la condició és:

`municipis['NOM'] == 'Palma'`

In [ ]:
# municipis[municipis['NOM'] == 'Palma']
municipis[municipis['name'] == 'accidents-causes-gu-bcn']

Podem emprar el mètode isin per seleccionar entre una llista de valors. Per exemple, si volem els municipis de les illes d'Eivissa i Formentera:

In [ ]:
# municipis[ municipis['ILLA'].isin( ['Eivissa', 'Formentera']) ]
municipis[ municipis['date'].isin( ['2015-11-19', '2015-06-10']) ]

Vegem els municipis amb més de 100 km2 de superfície (10.000 hectàrees)

In [ ]:
# municipis[municipis['HECTAREES'] > 10000]
municipis[municipis['package_count'] > 50]

Podem emprar els operadors lògics & (i lògica), | (o lògica) per a especificar múltiples condicions. Per exemple, els que tenen més de 100 km2 però menys de 200

In [ ]:
# municipis[(municipis['HECTAREES'] > 10000) & (municipis['HECTAREES'] < 20000)]
municipis[(municipis['package_count'] > 50) & (municipis['package_count'] < 70)]

El mètode where està molt relacionat, vegem com funciona:

In [ ]:
# municipis.where(municipis['HECTAREES'] > 10000)
municipis.where(municipis['package_count'] > 50)

També tenim el mètode query que ens permet especificar la condició com una cadena de text de la forma

`columna operador valor`

In [ ]:
# municipis.query('HECTAREES > 10000')
municipis.query('package_count > 50')

## Operacions d'ordenació

Per ordenar un dataframe per una columna, podem emprar el mètode sort_values, especificant amb l'argument by el nom de la columna. Per exemple, per ordenar alfabèticament els nostres municipis:

In [ ]:
# municipis.sort_values(by = 'NOM', ascending=False)
municipis.sort_values(by = 'date', ascending=False)

Igual que amb altres mètodes, això no afecta el dataframe original. Podem emprar inplace=True o una assignació si volem canviar l'ordre. De totes formes, això no modificarà els índexs.

Podem especificar diverses columnes, mitjançant una llista en lloc d'un únic nom de columna.

També podem especificar si volem l'ordre invers amb l'argument ascending=False.

Fins i tot podem determinar quin algorisme es farà servir per a l'ordenació (quicksort, mergesort o heapsort), mitjançant l'atribut kind.

Per exemple, vegem com ordenar de més superfície a menys.

In [ ]:
# municipis.sort_values(by = 'HECTAREES', ascending=False)
municipis.sort_values(by = 'package_count', ascending=False)

## Operacions d'agrupació
Ens queda una operació molt habitual quan analitzam dades: agrupar els valors comuns d'una columna. Per exemple, volem saber la superfície total de cada illa (la suma de tots els seus municipis):

* Primer, seleccionam només les columnes que han d'intervenir: ILLA i HECTAREES.
* Segon, agrupam per ILLA mitjançant el mètode groupby.
* Tercer, aplciam una funció d'agregació, en aquest cas, la suma.

In [ ]:
# municipis[['ILLA', 'HECTAREES']].groupby("ILLA").sum()
municipis[['date', 'package_count']].groupby("date").sum()

Podem utilitzar altres funcions d'agregació com la mitjana (mean) o el número d'instàncies (count)

In [ ]:
# municipis[['ILLA', 'HECTAREES']].groupby("ILLA").count().rename(columns={'HECTAREES': 'NOMBRE DE MUNICIPIS'})
municipis[['date', 'package_count']].groupby("date").count().rename(columns={'package_count': 'COUNT'})

El mètode groupby no retorna un dataframe, sinó un objecte pandas.api.typing.DataFrameGroupBy. Podem accedir a l'array amb els valors del seu índex (les illes en aquest cas):

In [ ]:
# municipis_illes = municipis[['ILLA', 'HECTAREES']].groupby("ILLA").count().rename(columns={'HECTAREES': 'NOMBRE DE MUNICIPIS'})
municipis_mod = municipis[['date', 'package_count']].groupby("date").count().rename(columns={'package_count': 'COUNT'})
municipis_mod.index.values

I a l'array amb els seus valors (el nombre de municipis en aquest cas):

In [ ]:
# municipis_illes.values
municipis_mod.values

## Operacions d'escriptura de dades
De la mateixa manera que podem carregar les dades d'un fitxer a un dataframe, podem fer l'operació inversa. Per exemple, el mètode to_csv exporta un dataframe a un fitxer CSV.

Per fer-ho des de Colab, hem de muntar la nostra unitat de Google Drive. En l'exemple, es crea el fitxer municipis.csv al directori dades (ha d'existir prèviament) de la meva unitat de Drive.

In [ ]:
#montam la nostra unitat de Google Drive
from google.colab import drive
drive.mount('/drive', force_remount=True)
municipis.to_csv('/drive/My Drive/dades/municipis.csv')

## Aclariment final: Series
Tot i que aquí hem treballat amb DataFrame, la majoria d'operacions que hem vist (excepte, lògicament, les que fan referència a columnes), es poden aplicar també a Series.

De fet, algunes d'elles, com les estadístiques, les hem aplicat directament sobre una columna, que és, en realitat, un objecte Series.

Per exemple, anam a obtenir un objecte Series a partir de la columna HECTAREES i calcular-ne la mitjana.

In [ ]:
superficies = municipis['HECTAREES']
type(superficies)

In [ ]:
superficies.mean()